In [1]:
import polars as pl
import requests

In [2]:
url = 'https://datos.madrid.es/egob/catalogo/300392-11041819-meteorologia-tiempo-real.csv'

dfp = pl.read_csv(url, separator=';', encoding='utf8')

dfp.shape

(87, 56)

In [3]:
dfp.head()

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,H02,V02,H03,V03,H04,V04,H05,V05,H06,V06,H07,V07,H08,V08,H09,V09,H10,V10,H11,V11,H12,V12,H13,V13,H14,V14,H15,V15,H16,V16,H17,V17,H18,V18,H19,V19,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
i64,i64,i64,i64,str,i64,i64,i64,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,i64,str,i64,str
28,79,102,81,"""28079102_81_98""",2024,12,14,0.47,"""V""",0.2,"""V""",0.48,"""V""",0.35,"""V""",0.3,"""V""",0.57,"""V""",0.57,"""V""",1.05,"""V""",1.17,"""V""",1.08,"""V""",1.05,"""V""",1.12,"""V""",1.07,"""V""",1.1,"""V""",0.98,"""V""",0.62,"""V""",0.47,"""V""",0.27,"""V""",0.28,"""V""",0.62,"""V""",0.37,"""V""",0.25,"""N""",0,"""N""",0,"""N"""
28,79,102,82,"""28079102_82_98""",2024,12,14,148.0,"""V""",139.0,"""V""",72.0,"""V""",53.0,"""V""",84.0,"""V""",58.0,"""V""",44.0,"""V""",68.0,"""V""",70.0,"""V""",79.0,"""V""",67.0,"""V""",41.0,"""V""",52.0,"""V""",78.0,"""V""",62.0,"""V""",101.0,"""V""",113.0,"""V""",268.0,"""V""",258.0,"""V""",11.0,"""V""",64.0,"""V""",131.0,"""N""",0,"""N""",0,"""N"""
28,79,102,83,"""28079102_83_98""",2024,12,14,2.1,"""V""",1.4,"""V""",0.5,"""V""",0.2,"""V""",0.4,"""V""",-0.2,"""V""",-0.6,"""V""",-1.2,"""V""",-1.5,"""V""",-1.8,"""V""",-1.1,"""V""",-0.3,"""V""",0.2,"""V""",0.7,"""V""",2.3,"""V""",4.8,"""V""",4.9,"""V""",4.2,"""V""",3.1,"""V""",3.0,"""V""",2.7,"""V""",2.4,"""N""",0,"""N""",0,"""N"""
28,79,102,86,"""28079102_86_98""",2024,12,14,86.0,"""V""",89.0,"""V""",91.0,"""V""",92.0,"""V""",90.0,"""V""",91.0,"""V""",93.0,"""V""",95.0,"""V""",97.0,"""V""",97.0,"""V""",98.0,"""V""",97.0,"""V""",97.0,"""V""",96.0,"""V""",88.0,"""V""",72.0,"""V""",68.0,"""V""",68.0,"""V""",76.0,"""V""",77.0,"""V""",78.0,"""V""",79.0,"""N""",0,"""N""",0,"""N"""
28,79,102,87,"""28079102_87_98""",2024,12,14,946.0,"""V""",946.0,"""V""",946.0,"""V""",946.0,"""V""",946.0,"""V""",947.0,"""V""",947.0,"""V""",947.0,"""V""",948.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",949.0,"""V""",950.0,"""V""",950.0,"""V""",950.0,"""N""",0,"""N""",0,"""N"""


In [6]:
dfp1 = dfp.filter(ESTACION=8, MAGNITUD = 83 ) #estacion escuales aguirre, temperatura

dfp2 =dfp1.unpivot(index = ['PROVINCIA',	'MUNICIPIO',	'ESTACION',	'MAGNITUD'	,'PUNTO_MUESTREO',	'ANO',	'MES','DIA'])

dfp3 = dfp2.with_columns((pl.col('variable').str.slice(0,1)).alias('NV')).with_columns((pl.col('variable').str.slice(1)).alias('HORA'))

dfp4 =(dfp3
    .pivot(on='NV', values='value', index = ['PROVINCIA',	'MUNICIPIO',	'ESTACION',	'MAGNITUD'	,'PUNTO_MUESTREO',	'ANO',	'MES',	'DIA', 'HORA'])
)

dfp5 = dfp4.with_columns(H = dfp4['H'].cast(float) ).with_columns(HORA = dfp4['HORA'].cast(int))

dfp6 = dfp5.with_columns( (pl.date(pl.col('ANO'),pl.col('MES'),pl.col('DIA') )).alias('DATE'))


dfp7 = dfp6.with_columns( (pl.datetime(year =pl.col('ANO'), month=pl.col('MES'),day=pl.col('DIA') ,hour=pl.col('HORA')-1)).alias('DATETIME'))
dfp7

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,HORA,H,V,DATE,DATETIME
i64,i64,i64,i64,str,i64,i64,i64,i64,f64,str,date,datetime[μs]
28,79,8,83,"""28079008_83_98""",2024,12,14,1,5.2,"""V""",2024-12-14,2024-12-14 00:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,2,4.9,"""V""",2024-12-14,2024-12-14 01:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,3,4.4,"""V""",2024-12-14,2024-12-14 02:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,4,3.9,"""V""",2024-12-14,2024-12-14 03:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,5,3.1,"""V""",2024-12-14,2024-12-14 04:00:00
…,…,…,…,…,…,…,…,…,…,…,…,…
28,79,8,83,"""28079008_83_98""",2024,12,14,20,6.1,"""V""",2024-12-14,2024-12-14 19:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,21,5.8,"""V""",2024-12-14,2024-12-14 20:00:00
28,79,8,83,"""28079008_83_98""",2024,12,14,22,5.0,"""V""",2024-12-14,2024-12-14 21:00:00


In [7]:
import hvplot.polars

dfp7.filter(pl.col('V')=='V').hvplot.line(x="DATETIME", y ="H")

:Curve   [DATETIME]   (H)